# Federated PCA  with Flex  for Anomaly Detection

In [1]:
from flexanomalies.models import PCA_Anomaly
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.decorators_pca import build_server_model_pca, copy_model_to_clients_pca,train_pca, aggregate_pca, set_aggregated_weights_pca, get_clients_weights_pca
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool

In [2]:
model_params = {
        "n_components": 4,
        "contamination": 0.1
}

In [4]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat("flexanomalies/datasets/data/shuttle.mat", 0.3)

In [5]:
model = PCA_Anomaly(**model_params)

In [6]:
flex_dataset = federate_data(5, X_train, y_train)

In [7]:
pool = FlexPool.client_server_pool(
        fed_dataset=flex_dataset,
        server_id="pca_server",
        init_func=build_server_model_pca,
        model=model,
    )

In [8]:
for i in range(3):
        print(f"\nRunning round: {i}\n")
        pool.servers.map(copy_model_to_clients_pca, pool.clients)
        pool.clients.map(train_pca)
        pool.aggregators.map(get_clients_weights_pca, pool.clients)
        pool.aggregators.map(aggregate_pca)
        pool.aggregators.map(set_aggregated_weights_pca, pool.servers)
output_model = pool.servers._models["pca_server"]["model"]
output_model.evaluate(X_test, y_test)


Running round: 0

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 1

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.

Running round: 2

Training model at client.
Training model at client.
Training model at client.
Training model at client.
Training model at client.
Acc: 96.477% 

Precision: 0.666 

F1score: 0.791 

Recall: 0.973 

AUC_ROC: 0.969 



In [9]:
output_model.result_metrics_

{'Accuracy': '96.477%',
 'Precision': '0.666',
 'F1': '0.791',
 'Recall': '0.973',
 'AUC_ROC': '0.969'}

In [10]:
save_experiments_results(
    'PCA',
    output_model,
    'test_PCA_notebook',
    model_params,
    'shuttle.mat',
    5,
    3,
    0.3,
)